In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import dlib
import argparse
import operator
import cv2
import glob
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adagrad

In [2]:
dataset = '/Users/dustint/Documents/Face Charisma Project/SuccessVsNonsuccess'
data_dir = pathlib.Path(dataset)

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

In [4]:
def face_alignment(file_path):
    image = cv2.imread(file_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # detect faces in the image
    rects = detector(gray, 2)
    if len(rects) == 0:
        return
    
    faceWidth = 224
    # face rect
    rect = rects[0]
    # both eyes and nose
    # left eye: 0,1 right eye: 2, 3 nose: 4
    shape = predictor(gray, rect)
    coords = np.zeros((5, 2), dtype=np.int)
    for i in range(5):
        coords[i] = (shape.part(i).x, shape.part(i).y)
        
    LEye = coords[:2].mean(axis=0).astype(np.int)
    REye = coords[2:4].mean(axis=0).astype(np.int)
    
    # calculate angle
    dY = REye[1] - LEye[1]
    dX = REye[0] - LEye[0]
    angle = np.degrees(np.arctan2(dY, dX)) - 180
    
    LRatio, RRatio = 0.35, 0.65
    eyeDist = np.sqrt(dX ** 2 + dY ** 2)
    normDist = 0.3 * faceWidth
    scale = normDist / eyeDist
    Center = (int(LEye[0] + REye[0]) // 2, int(LEye[1] + REye[1]) // 2)
    
    M = cv2.getRotationMatrix2D(Center, angle, scale)
    
    tX = faceWidth * 0.5
    tY = faceWidth * LRatio 
    M[0, 2] += (tX - Center[0])
    M[1, 2] += (tY - Center[1])
    
    #faceAligned = cv2.warpAffine(image, M, (faceWidth, faceWidth), cv2.INTER_CUBIC)
    
    new_image = np.zeros(image.shape, image.dtype)
    
    faceAligned = cv2.warpAffine(image, M, (faceWidth, faceWidth), cv2.INTER_CUBIC)
    return faceAligned

In [7]:
dir_path = '/Users/dustint/Documents/Face Charisma Project/SuccessVsNonsuccess'
out_dir_path = '/Users/dustint/Documents/Face Charisma Project/SuccessVsNonsuccessProcessed'

for path, _, files in os.walk(dir_path):
    for name in files:
        if name.endswith('.jpg' or '.png'):
            file_path = os.path.join(path, name)
            #print(file_path)
            
            proc_image = face_alignment(file_path)
            if proc_image is None:
                # Preprocessed failed, just skip
                continue
            
            
            new_file_path = file_path.replace('SuccessVsNonsuccess', 'SuccessVsNonsuccessProcessed')
            cv2.imwrite(new_file_path, proc_image)

/var/folders/4h/7fcfpb6d4w12gg7ydf_13wr40000gn/T/ipykernel_45771/3551350882.py:16: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  coords = np.zeros((5, 2), dtype=np.int)
/var/folders/4h/7fcfpb6d4w12gg7ydf_13wr40000gn/T/ipykernel_45771/3551350882.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your